In [89]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
pd.set_option('display.max_columns', 500)

In [6]:
import os
os.listdir('../../data')

['categories.csv',
 'products.csv',
 'orders.csv',
 'customer.csv',
 'department.csv',
 'order_items.csv']

In [7]:
Categories = pd.read_csv('../../data/categories.csv', delimiter=',')
Products = pd.read_csv('../../data/products.csv', delimiter=';')
Orders = pd.read_csv('../../data/orders.csv', delimiter=';')
Customer = pd.read_csv('../../data/customer.csv', delimiter=';')
Department = pd.read_csv('../../data/department.csv', delimiter=';')
Order_Items = pd.read_csv('../../data/order_items.csv', delimiter=';')

In [8]:
del Customer['customer_email']
del Customer['customer_password']
Customer.head(2)

,customer_id,customer_fname,customer_lname,customer_street,customer_city,customer_state,customer_zipcode
0,1,Richard,Hernandez,6303 Heather Plaza,Brownsville,TX,78521
1,2,Mary,Barrett,9526 Noble Embers Ridge,Littleton,CO,80126


In [9]:
Department.columns.values

array(['department_id', 'department_name'], dtype=object)

In [10]:
Order_Items.columns.values

array(['order_item_id', 'order_item_order_id', 'order_item_product_id',
       'order_item_quantity', 'order_item_subtotal',
       'order_item_product_price'], dtype=object)

In [11]:
def f(startsWith = ''):
    def g(val):
        if(val == startsWith + 'id'):
            return val
        return val.replace(startsWith, '')
    return g

FeatureMatrix = pd.DataFrame.from_dict({
    'Categories': np.vectorize(f('category_'))(Categories.columns.values).tolist(),
    'Products': np.vectorize(f('product_'))(Products.columns.values).tolist(),
    'Orders': np.vectorize(f('order_'))(Orders.columns.values).tolist(),
    'Customer': np.vectorize(f('customer_'))(Customer.columns.values).tolist(),
    'Department': np.vectorize(f('department_'))(Department.columns.values).tolist(),
    'Order_Items': np.vectorize(f('order_item_'))(Order_Items.columns.values).tolist()
}, orient='index').fillna(value = '-')
FeatureMatrix.head(10)

,0,1,2,3,4,5,6
Categories,category_id,department_id,name,-,-,-,-
Products,product_id,category_id,name,description,price,image,-
Orders,order_id,date,customer_id,status,-,-,-
Customer,customer_id,fname,lname,street,city,state,zipcode
Department,department_id,name,-,-,-,-,-
Order_Items,order_item_id,order_id,product_id,quantity,subtotal,product_price,-


We would like to pick up those attributes which answers any of the following question:
* Who purchased
* What purchased
* When purchased
* How much purchased

For the above question set we are picking up relevant feature for our feature matrix, this is totally based on business knowledge. Anything doubtful should be considered `in`.
* **Customer Id** - 
* **city** 
* **street** 
* **zipcode**
* **state**
* **ordered_product_name**
* **ordered_product_desciption**
* **ordered_product_price**
* **ordered_product_has_image**   (As of now, all products has image, we are just including this, but won't use this one) - `Products['product_image'].isnull().sum()`
* **order_date**
* **order_status**
* **department_name** - Not sure how helpful this will be, benefit of doubt goes to addition of attribute.
* **quantity**
* **subtotal**
* **product_price**

#### Biz problem
 - CEO wants to initiate an email marketting campain, so in order to get most out of it, he want to use our knowledge.
 
#### Thought process
 As a data analyst, let's try out a conversation between DA and consumer
 - **DA**: I have sent you a mail. Would you like to check it ?
 - User: Why should I?
 - **DA**: It may contains the items you are interested in?
 - User: May or does it ?
 - **DA**: `We need to figure out the list of items user is interested in`
 - *DA: We have some amazing offers for you?
 - User: Okay !!
 - DA: `We need to find out best combination of interested product and offer`[Out of our scope as dataset doesn't have any attribute for offers and how much to offer depends on business.]
 - User: This list seems to be providing good.

Now, we have following task in hand:
  - We need to figure out the list of items user is interested in
    - **Approach#1** - User might be interested in items in which he was interested in recent past.
    - **Approach#2** - User might be interested items which other similar users has bought. Ex - If i bought TV, then what are those items which other people has purchased along with or in near duration of purchasing a TV.
    - **Approach#3** - User might be offer alcholic - Ex Some users ends up buying lot unnecessary stuff during sale.

## Prepare dataset as required

In [94]:
TempDf1 = pd.merge(Orders, Customer, left_on='order_customer_id', right_on='customer_id')
Tempdf2 = pd.merge(TempDf1, Order_Items, left_on='order_id', right_on='order_item_order_id')
Tempdf3 = pd.merge(Tempdf2, Products, left_on='order_item_product_id', right_on='product_id')
Tempdf4 = pd.merge(Tempdf3, Categories, left_on='product_category_id', right_on='category_id')
Tempdf5 = pd.merge(Tempdf4, Department, left_on='category_department_id', right_on='department_id')
dfToProcess = Tempdf5[[  'customer_id', 
                         'customer_city', 
                         'customer_street', 
                         'customer_state',
                         'customer_zipcode',
                         'product_name',
                         'product_description',
                         'product_price',
                         'product_image',
                         'order_id',
                         'order_date',
                         'order_status',
                         'order_item_quantity',
                         'order_item_subtotal',
                         'order_item_product_price',
                         'category_name',
                         'department_name'
                      ]].copy(deep=True)
dfToProcess

,customer_id,customer_city,customer_street,customer_state,customer_zipcode,product_name,product_description,product_price,product_image,order_id,order_date,order_status,order_item_quantity,order_item_subtotal,order_item_product_price,category_name,department_name
0,11599,Hickory,8708 Indian Horse Highway,NC,28601,Diamondback Women's Serene Classic Comfort Bi,NaN,299.98,http://images.acmesports.sports/Diamondback+Wo...,1,2013-07-25 00:00:00,CLOSED,1,299.98,299.98,Camping & Hiking,Fan Shop
1,256,Chicago,7605 Tawny Horse Falls,IL,60625,Diamondback Women's Serene Classic Comfort Bi,NaN,299.98,http://images.acmesports.sports/Diamondback+Wo...,52893,2014-06-23 00:00:00,COMPLETE,1,299.98,299.98,Camping & Hiking,Fan Shop
2,12111,Santa Cruz,8766 Clear Prairie Line,CA,95060,Diamondback Women's Serene Classic Comfort Bi,NaN,299.98,http://images.acmesports.sports/Diamondback+Wo...,9017,2013-09-19 00:00:00,COMPLETE,1,299.98,299.98,Camping & Hiking,Fan Shop
3,12111,Santa Cruz,8766 Clear Prairie Line,CA,95060,Diamondback Women's Serene Classic Comfort Bi,NaN,299.98,http://images.acmesports.sports/Diamondback+Wo...,55826,2014-07-13 00:00:00,CLOSED,1,299.98,299.98,Camping & Hiking,Fan Shop
4,12111,Santa Cruz,8766 Clear Prairie Line,CA,95060,Diamondback Women's Serene Classic Comfort Bi,NaN,299.98,http://images.acmesports.sports/Diamondback+Wo...,56669,2014-07-18 00:00:00,PENDING_PAYMENT,1,299.98,299.98,Camping & Hiking,Fan Shop
5,8827,San Antonio,8396 High Corners,TX,78240,Diamondback Women's Serene Classic Comfort Bi,NaN,299.98,http://images.acmesports.sports/Diamondback+Wo...,3934,2013-08-17 00:00:00,COMPLETE,1,299.98,299.98,Camping & Hiking,Fan Shop
6,11318,Caguas,3047 Silent Embers Maze,PR,725,Diamondback Women's Serene Classic Comfort Bi,NaN,299.98,http://images.acmesports.sports/Diamondback+Wo...,5,2013-07-25 00:00:00,COMPLETE,1,299.98,299.98,Camping & Hiking,Fan Shop
7,11318,Caguas,3047 Silent Embers Maze,PR,725,Diamondback Women's Serene Classic Comfort Bi,NaN,299.98,http://images.acmesports.sports/Diamondback+Wo...,5,2013-07-25 00:00:00,COMPLETE,1,299.98,299.98,Camping & Hiking,Fan Shop
8,11318,Caguas,3047 Silent Embers Maze,PR,725,Diamondback Women's Serene Classic Comfort Bi,NaN,299.98,http://images.acmesports.sports/Diamondback+Wo...,27819,2014-01-13 00:00:00,PAYMENT_REVIEW,1,299.98,299.98,Camping & Hiking,Fan Shop
9,11318,Caguas,3047 Silent Embers Maze,PR,725,Diamondback Women's Serene Classic Comfort Bi,NaN,299.98,http://images.acmesports.sports/Diamondback+Wo...,29031,2014-01-20 00:00:00,ON_HOLD,1,299.98,299.98,Camping & Hiking,Fan Shop


## Approach#1 - Understanding user's interest